In [2]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from datetime import datetime
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
from plotly import figure_factory as FF
import folium
from folium import plugins

## Import Data

In [3]:
ny = pd.read_csv("analyze_me.csv")

## Exploratory Data Analysis

In [4]:
ny.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5983 entries, 0 to 5982
Data columns (total 18 columns):
delivery_id                            5983 non-null int64
customer_id                            5983 non-null int64
jumpman_id                             5983 non-null int64
vehicle_type                           5983 non-null object
pickup_place                           5983 non-null object
place_category                         5100 non-null object
item_name                              4753 non-null object
item_quantity                          4753 non-null float64
item_category_name                     4753 non-null object
how_long_it_took_to_order              3038 non-null object
pickup_lat                             5983 non-null float64
pickup_lon                             5983 non-null float64
dropoff_lat                            5983 non-null float64
dropoff_lon                            5983 non-null float64
when_the_delivery_started              5983 non-nul

In [5]:
ny.head()

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff
0,1457973,327168,162381,van,Melt Shop,American,Lemonade,1.0,Beverages,00:19:58.582052,40.744607,-73.990742,40.752073,-73.985370,2014-10-26 13:51:59.898924,NaN,NaN,2014-10-26 14:52:06.313088
1,1377056,64452,104533,bicycle,Prince Street Pizza,Pizza,Neapolitan Rice Balls,3.0,Munchables,00:25:09.107093,40.723080,-73.994615,40.719722,-73.991858,2014-10-16 21:58:58.65491,2014-10-16 22:26:02.120931,2014-10-16 22:48:23.091253,2014-10-16 22:59:22.948873
2,1476547,83095,132725,bicycle,Bareburger,Burger,Bare Sodas,1.0,Drinks,00:06:44.541717,40.728478,-73.998392,40.728606,-73.995143,2014-10-28 21:39:52.654394,2014-10-28 21:37:18.793405,2014-10-28 21:59:09.98481,2014-10-28 22:04:40.634962
3,1485494,271149,157175,bicycle,Juice Press,Juice Bar,OMG! My Favorite Juice!,1.0,Cold Pressed Juices,NaN,40.738868,-74.002747,40.751257,-74.005634,2014-10-30 10:54:11.531894,2014-10-30 11:04:17.759577,2014-10-30 11:16:37.895816,2014-10-30 11:32:38.090061
4,1327707,122609,118095,bicycle,Blue Ribbon Sushi,Japanese,Spicy Tuna & Tempura Flakes,2.0,Maki (Special Rolls),00:03:45.035418,40.726110,-74.002492,40.709323,-74.015867,2014-10-10 00:07:18.450505,2014-10-10 00:14:42.702223,2014-10-10 00:25:19.400294,2014-10-10 00:48:27.150595


In [6]:
ny.tail()

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff
5978,1360750,378035,151467,bicycle,Five Guys Burgers and Fries,Burger,Cheeseburger,1.0,Burgers,NaN,40.804404,-73.966430,40.818637,-73.939241,2014-10-14 13:30:25.567577,2014-10-14 13:41:09.42809,2014-10-14 13:53:28.264588,2014-10-14 14:10:19.256853
5979,1348697,96943,3296,bicycle,Cafe Mogador,Middle Eastern,Vegetarian,1.0,Cous Cous,NaN,40.727293,-73.984517,40.725938,-73.980550,2014-10-12 18:37:42.565897,2014-10-12 18:44:21.331886,2014-10-12 18:54:16.442017,2014-10-12 19:00:43.047785
5980,1274438,355090,153113,bicycle,Shake Shack,Burger,Fries,1.0,Fries,NaN,40.780826,-73.976483,40.763573,-73.973503,2014-10-01 18:12:56.973655,2014-10-01 18:29:59.803516,2014-10-01 18:52:19.769244,2014-10-01 19:16:52.653179
5981,1470282,400983,142140,car,Omai,Vietnamese,Ca Tim,1.0,Appetizers,00:02:17.264098,40.744408,-74.002891,40.734609,-74.006405,2014-10-27 20:25:03.592287,2014-10-27 20:22:37.822461,2014-10-27 20:39:25.770399,2014-10-27 20:50:45.259345
5982,1357449,128517,134189,car,RedFarm Broadway,Chinese,Pan-Fried Pork Buns (4),1.0,Dim Sum,00:09:25.288786,40.782117,-73.980739,40.732714,-73.995040,2014-10-13 20:30:44.473682,2014-10-13 20:40:22.494428,2014-10-13 21:06:02.105792,2014-10-13 21:31:54.396162


From the initial assessment of the data, we see that there are a total of 5983 deliveries recorded. And there are NaN values that need to be considered when applying functions over columns that have them.

### To analyze the market, we can start with asking questions that relate to what the customers would like and how the Jumpmen would best meet those needs. Then we can explore if the data provided could answer those questions(*). And if the data has information provided, what is the quality of that data. Data integrity would affect how much weight there is for conlusions drawn from data analysis.

### 3-sided market.

For the customers:
1. **Are there a lot of options?**
2. **Does the food get to me in a sufficient amount of time?** 
3. Is the food in good condition? [Accuracy of order] 
3. Is my courier cheerful? [Ratings, reviews, tips] 
4. Convenient payment option? [Checkout process (Is there friction leading to churn?)]

For the Jumpmen:
1. **Am I getting a lot of requests?** 
2. **Are all my requests located or mapped out in an efficient way?** 
3. **Are the food vendors able to provide food in a timely manner?**
4. **Is their vehicle type fit for food deliveries?**

For the food vendors/restaurants:
1. **Are there large volumes of orders?** 
2. Is the food quality good to last during delivery?
3. **Able to fulfill orders in a timely manner?**
______________________________________________________________________________________________________________________

For Jumpman23:
1. **Are there a lot of food vendors?**
2. **What is the volume of food that is ordered-- large?** 
2. **Customers happy?** 
3. **Couriers happy?**
4. Ratings, reviews and tips?

### The questions in bold are the ones that can be answered from our data. Now we can provide caveats about the data prior to making assessments of the market.

Where the missing data is located: In the 'place category', 'item name', 'item quantity' and 'item_category_name', 'how_long_it_took_to_order', 'when_the_Jumpman_arrived_at_pickup and 'when_the_Jumpman_left_pickup' columns. The 'place_category' is missing 883 records. The item categories are missing 1230 and the 'how_long_it_took_to_order is missing the largest amount of records- 2945. That is about 50% of the dataset. Lastly, 'when_the_Jumpman_arrived_at_pickup and 'when_the_Jumpman_left_pickup' columns are missing 550 records, which is the least amount, but still a significant number.

## Data Visualizations

In [7]:
vehicle = ny.vehicle_type.value_counts(normalize=0)
vehicle

bicycle       4274
car           1215
walker         274
van             76
scooter         75
truck           48
motorcycle      21
Name: vehicle_type, dtype: int64

In [8]:
pie_chart = [go.Pie(labels = vehicle.index, values = vehicle )]
py.iplot(go.Figure(data=pie_chart, layout = go.Layout(title= 'Jumpman Method of Transport')))

In [9]:
series1 = ny['pickup_place'].value_counts()
series1.head()

Shake Shack          311
Momofuku Milk Bar    186
The Meatball Shop    184
Blue Ribbon Sushi    151
sweetgreen           149
Name: pickup_place, dtype: int64

In [10]:
#Scroll over each bar :)

series1.iplot(kind='bar', yTitle='Count', title='Distribution of Pickup Places')

In [11]:
series2 = ny['place_category'].value_counts()
series2.head()

Italian     504
Burger      454
Japanese    433
American    405
Chinese     332
Name: place_category, dtype: int64

In [12]:
series2.iplot(kind='bar', yTitle='Count', title='Distribution of Place Category')

The following three columns that describe the items: item_name, item_quantity and item_category_name all are missing (1230/5983)100 = **20.55%** of the dataset. That means there is no information on the food requested. This could affect analysis of the market severely since 1/5 is a huge chunk. However, after looking at the distribution of data from all three bar graphs, there are clear high frequency item names, category names and quantities. Therefore, the missing data does not severely affect market analysis. Clear judgment on particular items/big sellers can still be made.

In [13]:
series3 = ny['item_name'].value_counts()
series3.head()

Fries           76
Cheese Fries    35
Shackburger     31
Chicken         30
Shack Burger    29
Name: item_name, dtype: int64

In [14]:
series3.iplot(kind='bar', yTitle='Count', title='Distribution of Item Names')

In [15]:
series4 = ny['item_category_name'].value_counts()
series4.head()

Sides         193
Burgers       148
Appetizers    145
Sandwiches    123
Fries         111
Name: item_category_name, dtype: int64

In [16]:
series4.iplot(kind='bar', yTitle='Count', title='Distribution of Item Category Names')

### How many orders are usually placed?

In [17]:
series5 = ny['item_quantity'].value_counts()
series5

1.0     3980
2.0      570
3.0      112
4.0       54
6.0       14
5.0       13
8.0        4
15.0       3
16.0       1
12.0       1
7.0        1
Name: item_quantity, dtype: int64

In [18]:
series5.iplot(kind='bar', yTitle='Count', title='Distribution of Quantity of Item Ordered')

### Based on the prior visualizations:

**Bicycle** is the most popular mode of transport. This makes sense in a huge metropolitan area as NYC where restaurants and homes are closer together.

**Shake Shack** is a hit with Momofuku Milk Bar and others far behind.

Though place_category is one of the incomplete columns/missing data, it is only missing 883 points (14.8% of the dataset). This is not as much as other columns, such as 'how_long_it_took_to_order', which is missing 2,945 points (49.2% of the dataset). Given, **Italian, Burger, Japanese, American and Chinese** are the top 5 requested.

**Fries** are the most popular.

**Sides** are the most desired.

Regarding how many orders are usually placed, it comes down to **one** order. Big orders are not very popular. However, this is suitable if the most popular modes of transportation for couriers are bicycle and car.

# Analysis of the Market

### CUSTOMER Side

How many customers used Jumpman23 more than once? Repeat customers would mean there is retention. This would mean Jumpman23 is offering an excellent product!

In [19]:
customer_groups_count = ny.groupby(['customer_id']).count()

In [20]:
customer_groups_count.sort_values(['delivery_id'],ascending = False).head(10)

,delivery_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff
customer_id,,,,,,,,,,,,,,,,,
369272,28,28,28,28,28,28,28,28,8,28,28,28,28,28,28,28,28
52832,23,23,23,23,22,23,23,23,17,23,23,23,23,23,23,23,23
275689,17,17,17,17,15,14,14,14,14,17,17,17,17,17,9,9,17
91817,16,16,16,16,13,13,13,13,8,16,16,16,16,16,12,12,16
58898,16,16,16,16,15,15,15,15,4,16,16,16,16,16,10,10,16
125123,16,16,16,16,15,12,12,12,5,16,16,16,16,16,13,13,16
100889,14,14,14,14,11,13,13,13,6,14,14,14,14,14,11,11,14
250494,13,13,13,13,10,12,12,12,5,13,13,13,13,13,11,11,13
115610,13,13,13,13,13,9,9,9,10,13,13,13,13,13,11,11,13


These are all the unique customers.

In [21]:
customer_groups_count.shape

(3192, 17)

 Let's see how many of these ordered >1 time.

In [22]:
customer_groups_count[customer_groups_count.delivery_id != 1].shape[0]

1253

There are **1253** customers who used Jumpman23 more than once. These are customers to examine what they like about Jumpman23. What would cause them to come back? Are they ordering from the same place? Are they getting their orders in a timely manner? Are they enjoying the couriers' awesome service?

For customers, what they like are a lot of options to choose from whether it be restaurants to menu items from each restaurant. Given how many pickup places, place categories, item names and item cateogory names there are, this could very well have been fulfiiled for customers who returned. 

Another criteria for customers to be satisfied would be given a reasonable amount of time for their food to get to them. This can be assessed by looking at the times it took to order, how long it took to start the pickup, pickup duration and the dropoff duration. Adding all these time intervals together would give us the time it took for the customer to get their food from tapping their mobile to make a request on their Postmates app to getting their food in hand.

In [23]:
ny['how_long_it_took_to_order']=pd.to_datetime(ny['how_long_it_took_to_order'])
ny['when_the_delivery_started']=pd.to_datetime(ny['when_the_delivery_started'])
ny['when_the_Jumpman_arrived_at_pickup']=pd.to_datetime(ny['when_the_Jumpman_arrived_at_pickup'])
ny['when_the_Jumpman_left_pickup']=pd.to_datetime(ny['when_the_Jumpman_left_pickup'])
ny['when_the_Jumpman_arrived_at_dropoff']=pd.to_datetime(ny['when_the_Jumpman_arrived_at_dropoff'])

In [24]:
ny.head(5)

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff
0,1457973,327168,162381,van,Melt Shop,American,Lemonade,1.0,Beverages,2019-03-19 00:19:58.582052,40.744607,-73.990742,40.752073,-73.985370,2014-10-26 13:51:59.898924,NaT,NaT,2014-10-26 14:52:06.313088
1,1377056,64452,104533,bicycle,Prince Street Pizza,Pizza,Neapolitan Rice Balls,3.0,Munchables,2019-03-19 00:25:09.107093,40.723080,-73.994615,40.719722,-73.991858,2014-10-16 21:58:58.654910,2014-10-16 22:26:02.120931,2014-10-16 22:48:23.091253,2014-10-16 22:59:22.948873
2,1476547,83095,132725,bicycle,Bareburger,Burger,Bare Sodas,1.0,Drinks,2019-03-19 00:06:44.541717,40.728478,-73.998392,40.728606,-73.995143,2014-10-28 21:39:52.654394,2014-10-28 21:37:18.793405,2014-10-28 21:59:09.984810,2014-10-28 22:04:40.634962
3,1485494,271149,157175,bicycle,Juice Press,Juice Bar,OMG! My Favorite Juice!,1.0,Cold Pressed Juices,NaT,40.738868,-74.002747,40.751257,-74.005634,2014-10-30 10:54:11.531894,2014-10-30 11:04:17.759577,2014-10-30 11:16:37.895816,2014-10-30 11:32:38.090061
4,1327707,122609,118095,bicycle,Blue Ribbon Sushi,Japanese,Spicy Tuna & Tempura Flakes,2.0,Maki (Special Rolls),2019-03-19 00:03:45.035418,40.726110,-74.002492,40.709323,-74.015867,2014-10-10 00:07:18.450505,2014-10-10 00:14:42.702223,2014-10-10 00:25:19.400294,2014-10-10 00:48:27.150595


NaN's trasformed into NaT's

In [25]:
ny['start_pickup_dur'] = ny.when_the_Jumpman_arrived_at_pickup - ny.when_the_delivery_started
ny['pickup_dur'] = ny.when_the_Jumpman_left_pickup - ny.when_the_Jumpman_arrived_at_pickup
ny['dropoff_dur'] = ny.when_the_Jumpman_arrived_at_dropoff - ny.when_the_Jumpman_left_pickup
ny['cust_wait_dur'] = ny['how_long_it_took_to_order'] + ny['start_pickup_dur'] + ny['pickup_dur'] + ny['dropoff_dur'] 

In [26]:
ny.head(5)

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,...,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff,start_pickup_dur,pickup_dur,dropoff_dur,cust_wait_dur
0,1457973,327168,162381,van,Melt Shop,American,Lemonade,1.0,Beverages,2019-03-19 00:19:58.582052,...,40.752073,-73.985370,2014-10-26 13:51:59.898924,NaT,NaT,2014-10-26 14:52:06.313088,NaT,NaT,NaT,NaT
1,1377056,64452,104533,bicycle,Prince Street Pizza,Pizza,Neapolitan Rice Balls,3.0,Munchables,2019-03-19 00:25:09.107093,...,40.719722,-73.991858,2014-10-16 21:58:58.654910,2014-10-16 22:26:02.120931,2014-10-16 22:48:23.091253,2014-10-16 22:59:22.948873,00:27:03.466021,00:22:20.970322,00:10:59.857620,2019-03-19 01:25:33.401056
2,1476547,83095,132725,bicycle,Bareburger,Burger,Bare Sodas,1.0,Drinks,2019-03-19 00:06:44.541717,...,40.728606,-73.995143,2014-10-28 21:39:52.654394,2014-10-28 21:37:18.793405,2014-10-28 21:59:09.984810,2014-10-28 22:04:40.634962,-1 days +23:57:26.139011,00:21:51.191405,00:05:30.650152,2019-03-19 00:31:32.522285
3,1485494,271149,157175,bicycle,Juice Press,Juice Bar,OMG! My Favorite Juice!,1.0,Cold Pressed Juices,NaT,...,40.751257,-74.005634,2014-10-30 10:54:11.531894,2014-10-30 11:04:17.759577,2014-10-30 11:16:37.895816,2014-10-30 11:32:38.090061,00:10:06.227683,00:12:20.136239,00:16:00.194245,NaT
4,1327707,122609,118095,bicycle,Blue Ribbon Sushi,Japanese,Spicy Tuna & Tempura Flakes,2.0,Maki (Special Rolls),2019-03-19 00:03:45.035418,...,40.709323,-74.015867,2014-10-10 00:07:18.450505,2014-10-10 00:14:42.702223,2014-10-10 00:25:19.400294,2014-10-10 00:48:27.150595,00:07:24.251718,00:10:36.698071,00:23:07.750301,2019-03-19 00:44:53.735508


In [27]:
ny['cust_wait_dur'].value_counts().head(20)

2019-03-19 00:34:18.935114    5
2019-03-19 01:21:56.529046    4
2019-03-19 01:17:33.240776    4
2019-03-19 00:40:00.460679    4
2019-03-19 00:51:24.380162    4
2019-03-19 00:52:40.483041    4
2019-03-19 01:10:25.300239    4
2019-03-19 01:41:13.208196    4
2019-03-19 01:39:30.485413    4
2019-03-19 01:03:44.119446    4
2019-03-19 00:36:01.894468    4
2019-03-19 00:55:29.236691    4
2019-03-19 02:23:37.427567    4
2019-03-19 01:04:18.367762    3
2019-03-19 01:07:35.053016    3
2019-03-19 00:42:33.927948    3
2019-03-19 00:58:39.958978    3
2019-03-19 00:33:52.517598    3
2019-03-19 00:56:59.148763    3
2019-03-19 01:08:25.937452    3
Name: cust_wait_dur, dtype: int64

The average wait time for a customer was apporoximately half an hour. This is not bad since some of the total wait durations are close to 2 hours...

### JUMPMAN Side

How many Jumpmen used Jumpman23 more than once? Repeat Jumpmen would mean there is retention in employment. This would mean Jumpman23 is offering competitive wages or demand (many restaurant vendors for customers to choose from)!

In [28]:
jumpman_groups_count = ny.groupby(['jumpman_id']).count()

In [29]:
jumpman_groups_count.sort_values(['delivery_id'],ascending = False).head(10)

,delivery_id,customer_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,pickup_lat,...,dropoff_lat,dropoff_lon,when_the_delivery_started,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff,start_pickup_dur,pickup_dur,dropoff_dur,cust_wait_dur
jumpman_id,,,,,,,,,,,,,,,,,,,,,
99219,78,78,78,78,70,64,64,64,55,78,...,78,78,78,70,70,78,70,70,70,52
104533,76,76,76,76,68,63,63,63,42,76,...,76,76,76,68,68,76,68,68,68,39
142394,73,73,73,73,64,53,53,53,42,73,...,73,73,73,64,64,73,64,64,64,38
66416,62,62,62,62,52,50,50,50,15,62,...,62,62,62,47,47,62,47,47,47,14
61900,61,61,61,61,58,57,57,57,53,61,...,61,61,61,49,49,61,49,49,49,43
30743,58,58,58,58,53,48,48,48,32,58,...,58,58,58,56,56,58,56,56,56,32
3296,57,57,57,57,44,46,46,46,27,57,...,57,57,57,55,55,57,55,55,55,25
32580,56,56,56,56,50,43,43,43,41,56,...,56,56,56,54,54,56,54,54,54,39
20962,56,56,56,56,48,43,43,43,35,56,...,56,56,56,55,55,56,55,55,55,35


In [30]:
jumpman_groups_count.shape 

(578, 21)

In [31]:
jumpman_groups_count[jumpman_groups_count.delivery_id != 1].shape[0]

485

There are **485** Jumpmen delivering multiple times. A couple of Jumpmen have even done it close to 80 times. This means they must like being part of Jumpman23! This sort of loyalty/positive attention helps in drawing more Jumpmen onto Jumpman23 platform.

One criteria for Jumpmen to be satisfied with their employment would be whether or not they are getting a lot of requests or put another way, if there are high demands. This is related to location of Jumpman to pickup places and also time of day. If a Jumpman is relatively close to a huge city and it is around lunch or dinner time on the weekend, then demand will be high. This can be assessed by looking at **Jumpmen's pickup places and order times during the day**. If there are high density areas and/or high density order times, then this would be attractive for Jumpmen wanting to get the most out of their Postmate runs.

Let's take a look at the density of pick_up places first with a heatmap using pick_up location coordinates.

In [32]:
#Here we set up the area in which we want to focus our coordinates and how much zoomed in we are on the map.
m = folium.Map([40.744607, -73.990742], zoom_start=12)
n = folium.Map([40.744607, -73.990742], zoom_start=12)

In [33]:
#Specifying Coordinates
pickup_loc = ny[['pickup_lat', 'pickup_lon']].values

# Plotting Heatmap
m.add_child(plugins.HeatMap(pickup_loc, radius=15))
m

This is a good sign for Jumpmen who want to focus his/her Postmates deliveries in high demand areas.

Now, let us look at high density order times.

In [34]:
def demand_hrs(row):
    return row.hour
ny.when_the_delivery_started.apply(demand_hrs).value_counts(normalize=1)

19    0.157112
18    0.125522
20    0.119505
17    0.076383
21    0.075882
12    0.068695
11    0.053986
13    0.053151
16    0.048304
14    0.044292
15    0.042286
22    0.035768
10    0.027578
23    0.023232
9     0.014708
0     0.012201
1     0.006686
8     0.005181
2     0.003677
7     0.003343
3     0.001170
4     0.001003
6     0.000334
Name: when_the_delivery_started, dtype: float64

In [35]:
ny.when_the_delivery_started.apply(demand_hrs).value_counts().iplot(kind='bar', title='Demand by Hour in Day')

We can see by this graph that most of the order times are happening between 17 hours and 21 hours (5pm-9pm).

Now let us take a look at where the food is going. It would be nice for Jumpman23 to see the food drop off locations relative to the food pickup locations to get a sense of travel distance needed.

In [36]:
dropoff_loc = ny[['dropoff_lat', 'dropoff_lon']].values

# Plotting Heatmap
n.add_child(plugins.HeatMap(dropoff_loc, radius=15))
n

Food pickup locations cover a wider area/not as concentrated. Relative to dropoff locations though, they do not seem too far apart. This means that Jumpmen are able to fulfill orders in a timley manner. Let us examine further time intervals during deliveries to see how the delivery service is running from time it takes order to be placed to time it takes for food to arrive at dropoff location. 

In [37]:
ny.start_pickup_dur[1].total_seconds()

1623.466021

In [38]:
def tot_sec(row):
    return row.total_seconds()
start_p = ny.start_pickup_dur.apply(tot_sec)
t_pick = ny.pickup_dur.apply(tot_sec)
t_drop = ny.dropoff_dur.apply(tot_sec)
ny['start_pickup_dur_m'] = round(start_p/60,2)
ny['pickup_dur_m'] = round(t_pick/60,2)
ny['dropoff_dur_m'] = round(t_drop/60,2)

In [39]:
ny[['start_pickup_dur_m','pickup_dur_m','dropoff_dur_m']].iplot(kind='box')

In [40]:
ny[['start_pickup_dur_m','pickup_dur_m','dropoff_dur_m']].describe()

,start_pickup_dur_m,pickup_dur_m,dropoff_dur_m
count,5433.000000,5433.000000,5433.000000
mean,12.709234,18.916033,14.135240
std,12.289227,12.447838,9.301326
min,-2.650000,0.000000,0.840000
25%,3.600000,10.820000,8.000000
50%,9.990000,15.810000,11.770000
75%,18.970000,23.840000,17.400000
max,142.090000,267.950000,119.190000


When looking at the mean values, time intervals seem reasonable. At the same time, we can see that the min time for start_pickup_dur_m has a negative value. This reveals that there is data integrity issues between 'when_the_delivery_started' and 'when_the_Jumpman_arrived_at_pickup'.

Last criteria for employee retention and also a metric of how well a food partner/restaurant is is whether or not food is provided in a timely manner.

## RESTAURANT side

Now, let us take a look at 'how_long_it_took_to_order' column. What we need to consider is that close to 50% of the data is missing. This means that we can only account for around 3000 orders made. This will suffice since this is only preliminary market data in NY. Given, there is going to be some gaps in the dataset. We will work with what we have.

This column is an INTERVAL. So this means that it is the time it took for the customer to request to the time it took for the food vendor/restaurant to confirm the order. Let us see what the average wait time was. This affects customer satisfaction and ultimately retention.

In [41]:
ny['how_long_it_took_to_order']=pd.to_datetime(ny['how_long_it_took_to_order'], format = '%H:%M:%S.%f')

Check if data got reformatted before performing aggregate functions.

In [42]:
type(ny['how_long_it_took_to_order'][0])

pandas._libs.tslibs.timestamps.Timestamp

Set up function to convert each delivery event's order time interval to total minutes. Then get overall statistics on the column including average, min, max, etc.

In [43]:
def conv_to_minutes(event):
    return event.minute*60.0 + event.second
min = ny['how_long_it_took_to_order'].apply(conv_to_minutes)
ny['how_long_it_took_to_order'] = round(min/60,2)

In [44]:
ny['how_long_it_took_to_order'].describe()

count    3038.000000
mean        7.751754
std         5.313122
min         1.370000
25%         4.430000
50%         6.280000
75%         9.170000
max        58.030000
Name: how_long_it_took_to_order, dtype: float64

In [45]:
series6 = ny['how_long_it_took_to_order'].value_counts()
series6.head()

3.67    16
4.90    14
4.33    14
4.57    14
3.93    14
Name: how_long_it_took_to_order, dtype: int64

In [46]:
series6.iplot(kind='bar', yTitle='Count', title='Distribution of How Long It Took to Place Order [intervals] in Minutes')

From here, we can see that average wait time for order to be placed is 7-8 minutes even though most of the time, it took between 3 and 4 minutes. This is not bad for food vendor/restaurant. This data does include the longest wait time a customer had which was close to an hour. Let's see if the customer completed the order and not cancelled by seeing if data exists for Jumpman arriving at drop off location. 

In [47]:
ny.loc[ny['how_long_it_took_to_order'] == 58.03]

,delivery_id,customer_id,jumpman_id,vehicle_type,pickup_place,place_category,item_name,item_quantity,item_category_name,how_long_it_took_to_order,...,when_the_Jumpman_arrived_at_pickup,when_the_Jumpman_left_pickup,when_the_Jumpman_arrived_at_dropoff,start_pickup_dur,pickup_dur,dropoff_dur,cust_wait_dur,start_pickup_dur_m,pickup_dur_m,dropoff_dur_m
5251,1449678,83112,60891,bicycle,Mile End Deli,Deli,Roast Beef,1.0,Sandwiches,58.03,...,2014-10-25 19:26:18.352273,2014-10-25 19:33:07.773520,2014-10-25 20:07:12.237977,00:59:20.940798,00:06:49.421247,00:34:04.464457,2019-03-19 02:38:16.944037,59.35,6.82,34.07


So even with a customer waiting close to an hour to place an order, order was not cancelled since there is a timestamp for when Jumpnan arrived at dropoff location. In this situation, there can be a coupon/free food given in order to prevent possible customer complaint and to preserve customer satisfaction. This can be done not just for this extreme case but for those outside of the standard deviation. Overall though, food vendors/restaurants did a good job at meeting order demand with supply.

# Jumpman23 as a whole - How Are Things Going in New York

Some things to consider as a food delivery service in a new market such as a metropolitan location as NYC is whether or not the demand is high, supply is high and if customers and Jumpmen are happy. And given the above mentioned market analysis of this preliminary datset, the market seems strong and growing. However, this needs to be taken with a grain of salt considering data integrity issues. The details to these follow.

Based on the timestamp data, there seems to be confusion with Jumpmen regarding when timestamp starts for delivery. Due to location of column, it seems to be the case that the intention of delivery start time is when person starts to head over to pickup place. However, some timestamps show time starting after Jumpmen arrives at pickup location. This is a data integrity issue. This would impact anaylsis because the correct set of data points would not be used for tracking trends for delivery start times and pickup arrival times. The other time stamps seem to be in order so analysis conclusions are stronger with remaining time stamps- when Jumpman arrived at pickup, left pickup and and arrived at dropoff location.

Lastly, most columns have missing values, which need to be made clear when drawing conclusions from data analysis.

In [48]:
ny.isnull().sum()

delivery_id                               0
customer_id                               0
jumpman_id                                0
vehicle_type                              0
pickup_place                              0
place_category                          883
item_name                              1230
item_quantity                          1230
item_category_name                     1230
how_long_it_took_to_order              2945
pickup_lat                                0
pickup_lon                                0
dropoff_lat                               0
dropoff_lon                               0
when_the_delivery_started                 0
when_the_Jumpman_arrived_at_pickup      550
when_the_Jumpman_left_pickup            550
when_the_Jumpman_arrived_at_dropoff       0
start_pickup_dur                        550
pickup_dur                              550
dropoff_dur                             550
cust_wait_dur                          3172
start_pickup_dur_m              

Original dataset had 7 out of 18 columns of missing data. Given, this needs to be taken into consideration when looking at summary statistics that may favor a positive outlook on what was a popular amout to order, popular food items, how long it took to order food or when Jumpmen seem to be doing stellar with their pickup times.

## Recommendations to CEO and CTO:

Before carrying on with new market in NY, would advise the following--

1. Eliminate erroneous data entries (empty/mis-recorded).
2. Reconfigure backend to restructure how data is gathered so from newly structured data storage unit, newly stored data is not as compromised as this preliminary dataset.
3. Not only relook at backend engineering, but also frontend by seeing how user experience can be modified to make ordering times faster/more optimized possibly with recommender system that stores customers' prior choices. This would allow less friction during customer journey and therefore smoother transition to checkout.